#Packages

In [ ]:
import yfinance as yf
import os
import pandas as pd
import numpy as np
import math
import datetime as dt
import requests
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [ ]:
url = 'https://min-api.cryptocompare.com/data/histoday'
params = {'fsym': 'BTC', 'tsym': 'USD', 'limit': 2000}

response = requests.get(url, params=params)
data = response.json()['Data']
df = pd.DataFrame(data)
df['time'] = pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace=True)

start = dt.datetime(2011, 1, 1)
end = dt.datetime.now()
delta = end - start

while delta.days > 0:
    if delta.days >= 2000:
        date = end - dt.timedelta(days=2000)
        params = {'fsym': 'BTC', 'tsym': 'USD', 'limit': 2000,
                  'toTs': date.timestamp()}
    else:
        params = {'fsym': 'BTC', 'tsym': 'USD', 'limit': delta.days,
          'toTs': df.index[0].timestamp()}
    response = requests.get(url, params=params)
    data = response.json()['Data']
    df1 = pd.DataFrame(data)
    df1['time'] = pd.to_datetime(df1['time'], unit='s')
    df1.set_index('time', inplace=True)
    delta -= dt.timedelta(days=2000)
    df = pd.concat([df1, df], axis=0)
btcData = pd.DataFrame(df)
btcData

#Check, Clean and Pre-process Data

In [ ]:
btcData = btcData.dropna()

In [ ]:
cols =  ["conversionType","conversionSymbol","volumefrom"]
btcData = btcData.drop(columns = cols,axis = 1)
btcData

#Model

In [ ]:
scaler = MinMaxScaler()
data_sc = scaler.fit_transform(btcData['close'].values.reshape(-1,1))

In [ ]:
prediction_days = 60
future_days = 30
x_train, y_train = [], []

for x in range(prediction_days, len(data_sc)-future_days):
    x_train.append(data_sc[x-prediction_days:x, 0])
    y_train.append(data_sc[x+future_days,0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train  =np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=30))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

In [ ]:
test_start = '2020-01-01'
test_data = btcData.iloc[btcData.index >= test_start].copy()
actual_price = test_data['close'].values

total_data = pd.concat([btcData['close'], test_data['close']])

model_inputs = total_data[len(total_data)-len(test_data)-prediction_days:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.fit_transform(model_inputs)

x_test = []

for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x,0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))

prediction_price = model.predict(x_test)
prediction_price = scaler.inverse_transform(prediction_price)

In [ ]:
real_data = [model_inputs[len(model_inputs) + 1 - prediction_days :len(model_inputs)+ 1, 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data,(real_data.shape[0],real_data.shape[1], 1))

prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)

prediction = np.reshape(prediction,(future_days))

In [ ]:
print(prediction[0])

26381.838
